In [120]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [121]:

df = pd.read_csv('data_half_price.csv')
print(f"Загружено: {len(df)} строк")

print(f"Средняя цена: {df['Price'].mean():,.0f} руб")
df.head()

Загружено: 22676 строк
Средняя цена: 18,060,661 руб


,Price,Apartment type,Metro station,Minutes to metro,Region,Number of rooms,Area,Living area,Kitchen area,Floor,Number of floors,Renovation
0,3150000.0,Secondary,Опалиха,6.0,Moscow region,1.0,30.6,11.1,8.5,25.0,25,Cosmetic
1,4500000.0,Secondary,Павшино,2.0,Moscow region,1.0,49.2,20.0,10.0,6.0,15,European-style renovation
2,5545000.0,Secondary,Мякинино,14.0,Moscow region,1.0,44.7,16.2,13.1,10.0,25,Cosmetic
3,4150000.0,Secondary,Строгино,8.0,Moscow region,1.0,35.1,16.0,11.0,12.0,33,European-style renovation
4,3225000.0,Secondary,Опалиха,6.0,Moscow region,1.0,37.7,15.2,4.0,5.0,5,Without renovation


In [122]:
# удаление выбросов (овермного или овермало)
initial_rows = len(df)
df = df[(df['Price'] >= 500_000) & (df['Price'] <= 200_000_000)]
print(f"Удалено выбросов по цене: {initial_rows - len(df)} строк")

initial_rows = len(df)
df = df[(df['Area'] >= 15) & (df['Area'] <= 300)]
print(f"Удалено выбросов по площади: {initial_rows - len(df)} строк")

initial_rows = len(df)
df = df[df['Floor'] <= df['Number of floors']]
print(f"Удалено некорректных этажей: {initial_rows - len(df)} строк")

print(f"\nИтоговый размер данных: {len(df)} строк")
print(f"Средняя цена после очистки: {df['Price'].mean():,.0f} руб")
print(f"Медианная цена: {df['Price'].median():,.0f} руб")

Удалено выбросов по цене: 207 строк
Удалено выбросов по площади: 763 строк
Удалено некорректных этажей: 1651 строк

Итоговый размер данных: 20055 строк
Средняя цена после очистки: 14,776,578 руб
Медианная цена: 5,900,000 руб


In [123]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# признаки по которым будем учиться
numeric_features = ['Area', 'Number of rooms', 'Floor', 'Number of floors', 'Minutes to metro']
X = df[numeric_features]
y = df['Price']

# разделяем данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nРазмер train: {len(X_train)} квартир")
print(f"Размер test: {len(X_test)} квартир")

# Самый простой бейзлайн: всегда предсказывает среднюю цену из train
# короче да, как я понял на любые параметры квартиры выдаст одну и ту же цену
mean_price = y_train.mean()
y_pred_dummy = np.full(len(y_test), mean_price)

# Оцениваем качество
mae_dummy = mean_absolute_error(y_test, y_pred_dummy)
rmse_dummy = np.sqrt(mean_squared_error(y_test, y_pred_dummy))
r2_dummy = r2_score(y_test, y_pred_dummy)

print(f"\n=== DUMMY МОДЕЛЬ (бейзлайн) ===")
print(f"Предсказывает всегда: {mean_price:,.0f} руб")
print(f"MAE: {mae_dummy:,.0f} руб")
print(f"RMSE: {rmse_dummy:,.0f} руб")
print(f"R²: {r2_dummy:.3f}")


Размер train: 16044 квартир
Размер test: 4011 квартир

=== DUMMY МОДЕЛЬ (бейзлайн) ===
Предсказывает всегда: 14,885,427 руб
MAE: 14,118,557 руб
RMSE: 24,208,889 руб
R²: -0.001


In [124]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# опять признаки, просто несколько раз случайно переопределил
numeric_features = ['Area', 'Number of rooms', 'Floor', 'Number of floors', 'Minutes to metro']
X = df[numeric_features]
y = df['Price']

# в целом опять то же самое разделение (просто полный код для регрессии с повторением что выше было)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Train: {len(X_train)} квартир, Test: {len(X_test)} квартир")

# нормализуем и масштабируем (чтобы не такая большая разница была у данных)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
y_train_scaled = scaler.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_test_scaled = scaler.transform(y_test.values.reshape(-1, 1)).flatten()

# 4. Обучаем модель (сама регрессия)
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# 5. Предсказываем (на тестовых данных)
y_pred_lr = lr_model.predict(X_test_scaled)

# 6. Оцениваем (считаем че как предсказала)
mae_lr = mean_absolute_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
r2_lr = r2_score(y_test, y_pred_lr)

print(f"\n=== ЛИНЕЙНАЯ РЕГРЕССИЯ ===")
print(f"MAE: {mae_lr:,.0f} руб")
print(f"RMSE: {rmse_lr:,.0f} руб")
print(f"R²: {r2_lr:.3f}")

# 7. Коэффициенты модели
print(f"\nКоэффициенты модели:")
for i, feature in enumerate(numeric_features):
    print(f"  {feature}: {lr_model.coef_[i]:,.0f} руб за единицу")
print(f"Базовая цена (intercept): {lr_model.intercept_:,.0f} руб")

Train: 16044 квартир, Test: 4011 квартир

=== ЛИНЕЙНАЯ РЕГРЕССИЯ ===
MAE: 7,899,754 руб
RMSE: 14,386,454 руб
R²: 0.647

Коэффициенты модели:
  Area: 23,110,088 руб за единицу
  Number of rooms: -3,911,521 руб за единицу
  Floor: -70,091 руб за единицу
  Number of floors: -947,111 руб за единицу
  Minutes to metro: -659,921 руб за единицу
Базовая цена (intercept): 14,885,427 руб


In [125]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [126]:
file_path = 'data_half_price.csv'
data = pd.read_csv(file_path)
print(data.head())

       Price Apartment type Metro station  Minutes to metro         Region  \
0  3150000.0      Secondary       Опалиха               6.0  Moscow region   
1  4500000.0      Secondary       Павшино               2.0  Moscow region   
2  5545000.0      Secondary      Мякинино              14.0  Moscow region   
3  4150000.0      Secondary      Строгино               8.0  Moscow region   
4  3225000.0      Secondary       Опалиха               6.0  Moscow region   

   Number of rooms  Area  Living area  Kitchen area  Floor  Number of floors  \
0              1.0  30.6         11.1           8.5   25.0                25   
1              1.0  49.2         20.0          10.0    6.0                15   
2              1.0  44.7         16.2          13.1   10.0                25   
3              1.0  35.1         16.0          11.0   12.0                33   
4              1.0  37.7         15.2           4.0    5.0                 5   

                  Renovation  
0                  

In [127]:
initial_rows = len(df)
df = df[(df['Price'] >= 500_000) & (df['Price'] <= 200_000_000)]
print(f"Удалено выбросов по цене: {initial_rows - len(df)} строк")

initial_rows = len(df)
df = df[(df['Area'] >= 15) & (df['Area'] <= 300)]
print(f"Удалено выбросов по площади: {initial_rows - len(df)} строк")

initial_rows = len(df)
df = df[df['Floor'] <= df['Number of floors']]
print(f"Удалено некорректных этажей: {initial_rows - len(df)} строк")

print(f"\nИтоговый размер данных: {len(df)} строк")
print(f"Средняя цена после очистки: {df['Price'].mean():,.0f} руб")
print(f"Медианная цена: {df['Price'].median():,.0f} руб")

Удалено выбросов по цене: 0 строк
Удалено выбросов по площади: 0 строк
Удалено некорректных этажей: 0 строк

Итоговый размер данных: 20055 строк
Средняя цена после очистки: 14,776,578 руб
Медианная цена: 5,900,000 руб


In [128]:


numerical_features = ['Area', 'Living area', 'Kitchen area', 'Number of rooms', 'Minutes to metro', 'Number of floors', 'Floor']
categorical_features = ['Apartment type', 'Metro station', 'Region', 'Renovation']
target = 'Price'

all_features = numerical_features + categorical_features

df = data[all_features + [target]].copy()
df.dropna(inplace=True)

X = df[all_features]
y = df[[target]].values
df.head()
# print(X)
# print(y)

,Area,Living area,Kitchen area,Number of rooms,Minutes to metro,Number of floors,Floor,Apartment type,Metro station,Region,Renovation,Price
0,30.6,11.1,8.5,1.0,6.0,25,25.0,Secondary,Опалиха,Moscow region,Cosmetic,3150000.0
1,49.2,20.0,10.0,1.0,2.0,15,6.0,Secondary,Павшино,Moscow region,European-style renovation,4500000.0
2,44.7,16.2,13.1,1.0,14.0,25,10.0,Secondary,Мякинино,Moscow region,Cosmetic,5545000.0
3,35.1,16.0,11.0,1.0,8.0,33,12.0,Secondary,Строгино,Moscow region,European-style renovation,4150000.0
4,37.7,15.2,4.0,1.0,6.0,5,5.0,Secondary,Опалиха,Moscow region,Without renovation,3225000.0


In [129]:
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(X, y, test_size=0.2, random_state=42)

scaler_x = StandardScaler()

# scaler_x.fit(X_train_raw)
# scaler_y.fit(y_train_raw)

X_train_scaled = scaler_x.fit_transform(X_train_raw[numerical_features])
X_test_scaled = scaler_x.transform(X_test_raw[numerical_features])

encoder_cat = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_train_cat_encoded = encoder_cat.fit_transform(X_train_raw[categorical_features])
X_test_cat_encoded = encoder_cat.transform(X_test_raw[categorical_features])

scaler_y = StandardScaler()
X_train_processed = np.hstack([X_train_scaled, X_train_cat_encoded])
X_test_processed = np.hstack([X_test_scaled, X_test_cat_encoded])

y_train_scaled = scaler_y.fit_transform(y_train_raw)
y_test_scaled = scaler_y.transform(y_test_raw)


# print(y_train_scaled)

In [130]:
X_train_tensor = torch.tensor(X_train_processed, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_processed, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_scaled, dtype=torch.float32)
# print(X_train_tensor)

In [131]:
class ApartmentPriceNet(nn.Module):
    def __init__(self, input_dim):
        super(ApartmentPriceNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)

In [132]:
input_dim = X_train_processed.shape[1]
model = ApartmentPriceNet(input_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [133]:
epochs = 1000

for epoch in range(epochs):
    model.train()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 0.0943
Epoch [200/1000], Loss: 0.0659
Epoch [300/1000], Loss: 0.0549
Epoch [400/1000], Loss: 0.0392
Epoch [500/1000], Loss: 0.0412
Epoch [600/1000], Loss: 0.0395
Epoch [700/1000], Loss: 0.0356
Epoch [800/1000], Loss: 0.0355
Epoch [900/1000], Loss: 0.0326
Epoch [1000/1000], Loss: 0.0311


In [134]:
model.eval()
with torch.no_grad():
    predicted_scaled = model(X_test_tensor).numpy()

    predicted_prices = scaler_y.inverse_transform(predicted_scaled)
    actual_prices = scaler_y.inverse_transform(y_test_scaled)

In [135]:
mae = mean_absolute_error(actual_prices, predicted_prices)
rmse = np.sqrt(mean_squared_error(actual_prices, predicted_prices))
r2 = r2_score(actual_prices, predicted_prices)

In [136]:
print("\n" + "="*50)
print("РЕЗУЛЬТАТЫ НА ТЕСТЕ (без data leakage):")
print(f"R2 Score: {r2:.4f}")
print(f"MAE: {mae:,.0f} руб.")
print(f"RMSE: {rmse:,.0f} руб.")
print("="*50)


РЕЗУЛЬТАТЫ НА ТЕСТЕ (без data leakage):
R2 Score: 0.7809
MAE: 4,112,975 руб.
RMSE: 19,051,016 руб.


In [137]:

print("\n" + "="*70)
print("ТЕСТОВОЕ ПРЕДСКАЗАНИЕ")
print("="*70)

model.eval()
with torch.no_grad():
    test_features = X_test_tensor[0:1]
    test_prediction = model(test_features).numpy()
    test_price = scaler_y.inverse_transform(test_prediction)[0][0]

    actual_price = scaler_y.inverse_transform(y_test_tensor[0:1].numpy())[0][0]

    print(f"Предсказанная цена: {test_price:,.0f} ₽")
    print(f"Реальная цена:      {actual_price:,.0f} ₽")
    print(f"Ошибка:             {abs(test_price - actual_price):,.0f} ₽")
    print(f"Относительная ошибка: {abs(test_price - actual_price)/actual_price*100:.1f}%")


ТЕСТОВОЕ ПРЕДСКАЗАНИЕ
Предсказанная цена: 3,870,580 ₽
Реальная цена:      2,100,000 ₽
Ошибка:             1,770,580 ₽
Относительная ошибка: 84.3%
